## Deploy model and create endpoint from keras h5 model file created in s3_transfer_learning_nursinghome.ipynb notebook

In [1]:
!pip install sagemaker

In [2]:
import tensorflow as tf
from tensorflow import keras
import sagemaker
import boto3, re
from sagemaker import get_execution_role
from PIL import Image
import numpy as np

2024-04-30 19:22:03.448029: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 19:22:03.489729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 19:22:03.489764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 19:22:03.490848: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 19:22:03.497231: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 19:22:03.497716: I tensorflow/core/platform/cpu_feature_guard.cc:1

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Load trained model from local directory

In [3]:
# load model from path, file full_resnet50_trained_model.h5
model = tf.keras.models.load_model('/home/sagemaker-user/project/models/resnet50_trained/full_resnet50_trained_model.h5')

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 tf.__operators__.getitem (  (None, 180, 180, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 180, 180, 3)       0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 6, 6, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0     

Convert model to .tar.gz format that is readable by aws sagemaker

In [5]:
def convert_model_to_aws(loaded_model):
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    
    tf.compat.v1.disable_eager_execution()
    
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

    with tf.compat.v1.Session() as sess:
        init = tf.compat.v1.global_variables_initializer()
        sess.run(init)
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()
    
    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_model_to_aws(model)

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


2024-04-30 19:22:09.409698: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


### warnings above ok
Uploading model to sagemake default bucket and deploying model and creating endpoint

In [6]:
# Uploading the model zip file to a default s3 bucket
import sagemaker
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

# Get the IAM role and specify the framework version
role = get_execution_role()
framework_version = '2.5' 

# Create Sagemaker model
!touch train.py
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = framework_version,
                                  entry_point = 'train.py')
# create endpoint - can take 5-10min
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.c5.4xlarge')

----!

In [7]:
print(predictor.endpoint)
print(predictor.endpoint_name)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


tensorflow-inference-2024-04-30-19-22-11-969
tensorflow-inference-2024-04-30-19-22-11-969


### As shown below by endpoint status, you can see endpoint is now InService and Active
Note: Delete Endpoint once Not in Use, via AWS Sagemaker Studio UI Endpoints Section to not incurr additional costs. 

In [9]:
# Create SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Describe endpoint
endpoint_description = sagemaker_client.describe_endpoint(EndpointName=predictor.endpoint_name)

# Get endpoint ARN and status
endpoint_arn = endpoint_description['EndpointArn']
endpoint_status = endpoint_description['EndpointStatus']

# Print or use the endpoint ARN and status as needed
print("SageMaker endpoint ARN:", endpoint_arn)
print("SageMaker endpoint status:", endpoint_status)


SageMaker endpoint ARN: arn:aws:sagemaker:us-east-1:975050027276:endpoint/tensorflow-inference-2024-04-30-19-22-11-969
SageMaker endpoint status: InService
